In [34]:
#import useful libraries
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
import os
from tensorflow.keras import losses
from tensorflow.keras import metrics

In [2]:
#directries
original_dir='/home/usama/Desktop/Aeroplane project'
dataset_dir='/home/usama/Desktop/Aeroplane project/areoplane dataset'

In [3]:
#import all cvs folder
train_dir=os.path.join(dataset_dir,'train.csv')
test_dir=os.path.join(dataset_dir,'test.csv')
sample_dir=os.path.join(dataset_dir,'sample_submission.csv')

In [4]:
#read all csv files
train_data_frame=pd.read_csv(train_dir)
test_data_frame=pd.read_csv(test_dir)
sample_submission_data_frame=pd.read_csv(sample_dir)

In [5]:
#check train data view
train_data_frame.head(5)

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14,22,71.285324,0.272118,78.04,2,31335.476824,3,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10,27,72.288058,0.423939,84.54,2,26024.711057,2,0.352350,12128
2,Significant_Damage_And_Fatalities,63.059361,13,16,66.362808,0.322604,78.86,7,39269.053927,3,0.003364,2181
3,Significant_Damage_And_Serious_Injuries,48.082192,11,9,74.703737,0.337029,81.79,3,42771.499200,1,0.211728,5946
4,Significant_Damage_And_Fatalities,26.484018,13,25,47.948952,0.541140,77.16,3,35509.228515,2,0.176883,9054


In [6]:
train_data_frame.describe()

,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000
mean,41.876406,12.931100,6.564300,65.145324,0.381495,79.969331,3.814900,32001.803282,2.01220,0.255635,6266.554200
std,16.138072,3.539803,6.971982,11.882934,0.121301,2.759739,1.902577,9431.995196,1.03998,0.381128,3610.170288
min,0.000000,1.000000,0.000000,0.000000,0.134000,74.740000,1.000000,831.695553,0.00000,0.000316,2.000000
25%,30.593607,11.000000,2.000000,56.927985,0.293665,77.960000,2.000000,25757.636908,1.00000,0.012063,3139.750000
50%,41.278539,13.000000,4.000000,65.587967,0.365879,79.540000,4.000000,32060.336419,2.00000,0.074467,6280.500000
75%,52.511416,15.000000,9.000000,73.336372,0.451346,81.560000,5.000000,38380.641513,3.00000,0.354059,9391.500000
max,100.000000,23.000000,54.000000,100.000000,0.882648,97.510000,7.000000,64297.651218,5.00000,2.365378,12500.000000


In [8]:
train_data_frame["Days_Since_Inspection"]=train_data_frame["Days_Since_Inspection"].astype(np.float16)
train_data_frame["Violations"]=train_data_frame["Violations"].astype(np.float16)
train_data_frame["Accident_Type_Code"]=train_data_frame["Accident_Type_Code"].astype(np.float16)
train_data_frame["Total_Safety_Complaints"]=train_data_frame["Total_Safety_Complaints"].astype(np.float16)

In [24]:
train_data_frame

,Severity,Safety_Score,Days_Since_Inspection,Total_Safety_Complaints,Control_Metric,Turbulence_In_gforces,Cabin_Temperature,Accident_Type_Code,Max_Elevation,Violations,Adverse_Weather_Metric,Accident_ID
0,Minor_Damage_And_Injuries,49.223744,14.0,22.0,71.285324,0.272118,78.04,2.0,31335.476824,3.0,0.424352,7570
1,Minor_Damage_And_Injuries,62.465753,10.0,27.0,72.288058,0.423939,84.54,2.0,26024.711057,2.0,0.352350,12128
2,Significant_Damage_And_Fatalities,63.059361,13.0,16.0,66.362808,0.322604,78.86,7.0,39269.053927,3.0,0.003364,2181
3,Significant_Damage_And_Serious_Injuries,48.082192,11.0,9.0,74.703737,0.337029,81.79,3.0,42771.499200,1.0,0.211728,5946
4,Significant_Damage_And_Fatalities,26.484018,13.0,25.0,47.948952,0.541140,77.16,3.0,35509.228515,2.0,0.176883,9054
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Significant_Damage_And_Serious_Injuries,56.118721,8.0,1.0,63.445761,0.482359,78.64,5.0,38537.363408,2.0,0.026150,11493
9996,Highly_Fatal_And_Damaging,40.365297,10.0,7.0,62.169553,0.421775,79.77,4.0,40846.916900,2.0,0.074898,305
9997,Significant_Damage_And_Serious_Injuries,27.853881,17.0,1.0,69.598906,0.318277,80.37,3.0,33771.996300,3.0,0.168173,612
9998,Minor_Damage_And_Injuries,56.210046,8.0,0.0,39.835916,0.598118,76.92,5.0,39820.419251,2.0,0.026381,4963


In [9]:
train_data=train_data_frame.iloc[:,1:11]
train_lable=train_data_frame.iloc[:,0]

In [10]:
mean=train_data.mean(axis=0)
train_data-=mean
std=train_data.std(axis=0)
train_data/=std

In [30]:
encoded=LabelEncoder()
encoded.fit(train_lable)
labels_encoded=encoded.transform(train_lable)
labels_encoded

array([1, 1, 2, ..., 3, 1, 0])

In [13]:
trainx,testx,trainy,testy=train_test_split(train_data,labels_encoded,test_size=0.25,random_state=42)

In [31]:
def build_model():
    model=Sequential()
    model.add(Dense(128,activation="relu",input_shape=(train_data.shape[1],)))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(16,activation='relu'))
    model.add(Dense(4,activation='softmax'))
    return model

In [35]:
model=build_model()
model.compile(optimizer=Adam(lr=1e-4),loss="sparse_categorical_crossentropy",metrics=['accuracy'])
model.fit(trainx,trainy,epochs=20,batch_size=1,validation_data=(testx,testy),verbose=1)

Train on 7500 samples, validate on 2500 samples
Epoch 1/20
7500/7500 [==============================] - 27s 4ms/sample - loss: 0.9736 - acc: 0.6260 - val_loss: 0.6814 - val_acc: 0.7776
Epoch 2/20
7500/7500 [==============================] - 26s 4ms/sample - loss: 0.4901 - acc: 0.8467 - val_loss: 0.4139 - val_acc: 0.8688
Epoch 3/20
7500/7500 [==============================] - 26s 4ms/sample - loss: 0.3223 - acc: 0.9037 - val_loss: 0.3255 - val_acc: 0.9016
Epoch 4/20
7500/7500 [==============================] - 27s 4ms/sample - loss: 0.2647 - acc: 0.9235 - val_loss: 0.2971 - val_acc: 0.9112
Epoch 5/20
7500/7500 [==============================] - 27s 4ms/sample - loss: 0.2385 - acc: 0.9291 - val_loss: 0.2744 - val_acc: 0.9192
Epoch 6/20
7500/7500 [==============================] - 30s 4ms/sample - loss: 0.2224 - acc: 0.9327 - val_loss: 0.2606 - val_acc: 0.9212
Epoch 7/20
7500/7500 [==============================] - 27s 4ms/sample - loss: 0.2075 - acc: 0.9348 - val_loss: 0.2511 - val_acc: 

In [36]:
model.save("aeroplane model")